# Pipline

Теперь вы можете объединить все части кода в единый алгоритм:

1. Тензорное модальное разложение (TBMD):
* Выполнить Tucker-разложение тензора 𝑋 для получения тензора 𝐴.

2. Оптимальное размещение сенсоров:
* Применить тензорное QR-разложение к 𝐴 для получения матрицы 𝑃.

3. Формирование измерений:
* Сформировать измерения 𝑌 с использованием 𝑃 и 𝑋.

4. Восстановление вектора весов 𝑥:
* Применить тензорное компрессивное измерение для восстановления 𝑥.

5. Восстановление поля 𝑋:
* Использовать 𝑥 и 𝐴 для восстановления 𝑋

# Download packages

In [1]:
%pip install -U tensorly torch

  Using cached torch-2.5.1-cp312-none-macosx_11_0_arm64.whl.metadata (28 kB)
  Using cached numpy-2.1.3-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached scipy-1.14.1-cp312-cp312-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached setuptools-75.3.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached MarkupSafe-3.0.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (4.0 kB)
Using cached torch-2.5.1-cp312-none-macosx_11_0_arm64.whl (63.9 MB)
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
Using cached typing_extensions-4.1

# Import libraries

In [68]:
import numpy as np
import tensorly as tl

from tensorly.decomposition import tucker
from tensorly import tenalg
from tensorly.tenalg import mode_dot
from typing import Set, Tuple, Union

In [3]:
tl.set_backend('numpy')

In [4]:
SEED = 42

tl.check_random_state(SEED)
np.random.seed(SEED)

# Download data

# Data

In [ ]:
# Параметры
I, J, K = 5, 5, 5
D = 3  # Количество наборов данных


# Генерируем D тензоров данных
X_tensor = np.random.rand(I, J, K)

# Applying TBMD to each tensor

Применяем тензорное модальное разложение (TBMD), в данном случае Tucker разложение, к каждому тензору, получая модальные тензоры 𝑀1, 𝑀2, …, 𝑀𝐷.

In [70]:
epsilon = 1e-5
ranks = [min(I, J, K) for _ in range(3)]

G_hat, factors = tucker(X_tensor, rank=ranks, tol=epsilon, random_state=SEED)

# Construction of the joint modal tensor 𝐴


Стекуем модальные тензоры 𝑀1, 𝑀2, …, 𝑀𝐷 по третьему измерению (или создаем объединенный тензор), получая тензор 𝐴.

In [42]:
# Предполагаем, что вы уже извлекли A_hat и B_hat из factors
A_hat, B_hat, C_hat = factors

In [43]:
M_list = []
for n in range(G_hat.shape[2]):
    G_slice = G_hat[:, :, n]  # Срез ядра по третьему измерению
    M_n = A_hat @ G_slice @ B_hat.T  # Матрица размера (I x J)
    M_list.append(M_n)

In [45]:
# Собираем модальные матрицы в тензор A
A_tensor = np.stack(M_list, axis=-1)  # Размерность будет (I, J, R), где R — число мод (ранг по третьему измерению)

# Application of Tensor QR Decomposition for Optimal Sensor Placement

Теперь нам нужно применить тензорное QR-разложение к тензору 𝐴 для получения матрицы перестановок 𝑃, которая будет использоваться для оптимального размещения сенсоров.

In [62]:
def tensor_based_tube_fiber_pivot_qr_factorization(R: tl.tensor, N: int) -> Tuple[tl.tensor, tl.tensor, tl.tensor]:
    """
    Implements the Tensor-based Tube Fiber-pivot QR Factorization.

    Parameters:
    - R: Input 3D tensor of shape (n1 x n2 x m), where n1 and n2 are dimensions of the matrix,
         and m is the size of each tube (depth).
    - N: Number of iterations for the factorization.
    - A: Set of already used indices (to avoid repetition in the pivot).
    - P: Output permutation matrix (n1 x n2) for selecting sensors or fibers.
    - Q: Orthogonal matrix to be updated (m x m).
    - M: Matrix to store the ℓ1-norms (n1 x n2) of the tubes from tensor R.

    Returns:
    - P: Updated permutation matrix with selected sensor placements.
    - Q: Updated orthogonal matrix after each iteration.
    - R: Updated tensor after applying Householder transformations.
    """

    n1, n2, m = R.shape
    P = np.zeros((n1, n2))  # Размерности матрицы P
    Q = np.eye(m)  # Предполагаем, что третья размерность соответствует времени или другому измерению
    M = np.zeros((n1, n2))
    A_set = set()


    for d in range(N):
        # Compute tubular ℓ1-norms and fill matrix M
        for i in range(n1):
            for j in range(n2):
                tube = R[i, j, :]
                M[i, j] = tl.norm(tube, 1)  # Using ℓ1-norm

        # Find the maximum element in M that has not been used
        while True:
            max_index = tl.argmax(M)
            max_index = int(max_index)
            x, y = divmod(max_index, n2)
            if (x, y) not in A_set:
                break
            else:
                M[x, y] = 0  # Zero out the element to avoid reusing it

        A_set.add((x, y))
        P[x, y] = 1  # Set the corresponding element in P to 1

        # Extract vector t from tensor R
        t = R[x, y, d:]

        # Compute sigma and vector u
        sigma = tl.norm(t, 2)
        if sigma == 0:
            u = tl.zeros_like(t)
        else:
            e_d = tl.zeros_like(t)
            e_d[0] = 1  # Position 0 corresponds to position d in Python
            t1 = t[0]
            sign_t1 = tl.sign(t1) if t1 != 0 else 1
            numerator = t + sign_t1 * sigma * e_d
            denominator = tl.sqrt(2 * sigma * (sigma + tl.abs(t1)))
            u = numerator / denominator

        # Update R for slice x, y, d:m
        R_slice = R[x, y, d:]
        R[x, y, d:] = R_slice - 2 * u * tl.dot(u, R_slice)

        # Update Q matrix
        u_Q = u.reshape(-1, 1)  # Reshape u to (m - d, 1)
        Q_d = Q[:, d:]  # Get submatrix of Q from column d onwards
        Q[:, d:] = Q_d - 2 * Q_d @ (u_Q @ u_Q.T)  # Update Q

    # Return the updated P, Q, and R matrices
    return P, Q, R

In [63]:
# def tensor_based_tube_fiber_pivot_qr_factorization(R, N):
#     n1, n2, m = R.shape
#     P = np.zeros((n1, n2))
#     Q = np.eye(m)
#     M = np.zeros((n1, n2))
#     A_set = set()
    
#     for d in range(N):
#         # Вычисляем ℓ1-нормы трубок
#         for i in range(n1):
#             for j in range(n2):
#                 tube = R[i, j, d:]
#                 M[i, j] = np.linalg.norm(tube, 1)
                
#         # Находим максимальную норму
#         while True:
#             max_index = np.argmax(M)
#             x, y = divmod(max_index, n2)
#             if (x, y) not in A_set:
#                 break
#             else:
#                 M[x, y] = 0
        
#         A_set.add((x, y))
#         P[x, y] = 1
        
#         # Обновляем R и Q
#         t = R[x, y, d:]
#         sigma = np.linalg.norm(t)
#         if sigma == 0:
#             u = np.zeros_like(t)
#         else:
#             e_d = np.zeros_like(t)
#             e_d[0] = 1
#             sign_t1 = np.sign(t[0]) if t[0] != 0 else 1
#             u = (t + sign_t1 * sigma * e_d) / np.sqrt(2 * sigma * (sigma + abs(t[0])))
        
#         # Обновляем R
#         R_slice = R[:, :, d:]
#         R[:, :, d:] = R_slice - 2 * np.tensordot(u, np.tensordot(u, R_slice, axes=([0], [2])), axes=([0], [0]))
        
#         # Обновляем Q
#         u_Q = u.reshape(-1, 1)
#         Q_d = Q[:, d:]
#         Q[:, d:] = Q_d - 2 * Q_d @ (u_Q @ u_Q.T)
        
#     return P, Q, R

In [64]:
# Параметры для QR-разложения
N = 5  # Задайте количество сенсоров

# Применяем функцию (не забудьте адаптировать функцию под ваши данные)
P, Q, R = tensor_based_tube_fiber_pivot_qr_factorization(A_tensor, N)

In [65]:
Q_T_Q = tl.dot(Q.T, Q)
identity = tl.tensor(np.eye(A_tensor.shape[2]), dtype=tl.float32)
difference = tl.norm(Q_T_Q - identity)


print(f"\nNorm of (Q^T Q - I): {difference}")


Norm of (Q^T Q - I): 7.805874596144724e-16


# Formation of dimensions 𝑌


После получения матрицы 𝑃 вы можете сформировать матрицу измерений 𝑌 из исходного тензора 𝑋.

In [69]:
# Получаем индексы выбранных сенсоров
sensor_indices = np.argwhere(P == 1)

Y_list = []

for idx in sensor_indices:
    i, j = idx
    # Извлекаем временной ряд данных для позиции сенсора (i, j)
    time_series = X_tensor[i, j, :]  # Размерность (K,)
    Y_list.append(time_series)

# Преобразуем список в numpy-массив
Y = np.array(Y_list)  # Размерность (N, K), где N — количество сенсоров

# Tensor compression measurement

Теперь нужно решить оптимизационную задачу для восстановления вектора весов 𝑥, используя измерения 𝑌 и тензор 𝐴.

In [73]:
def tensor_based_compressive_sensing(
    A: Union[np.ndarray, tl.tensor],
    P: Union[np.ndarray, tl.tensor],
    Y: Union[np.ndarray, tl.tensor],
    max_iter: int,
    epsilon: float,
    lambd: float,
    delta_0: float,
    delta_max: float
) -> tl.tensor:
    """
    Implements the Tensor-based Compressive Sensing Algorithm (Algorithm 3).
    
    Parameters:
    - A: Input tensor of shape (I x J x W) (numpy array or Tensorly tensor)
    - P: Permutation matrix or sensor selection matrix (should be compatible for mode-0 multiplication with A)
         (numpy array or Tensorly tensor)
    - Y: Observed data tensor (after sensor placement) (numpy array or Tensorly tensor)
    - max_iter: Maximum number of iterations (int)
    - epsilon: Small positive scalar for shrinkage thresholding (float)
    - lambd: Regularization parameter (float)
    - delta_0: Initial value for the Lagrange penalty term (float)
    - delta_max: Maximum value for delta (float)

    Returns:
    - x_hat: Recovered sparse code vector of shape (W x 1) (Tensorly tensor)
    """

    # Ensure A, P, Y are tensors
    A = tl.tensor(A, dtype=tl.float32)
    P = tl.tensor(P, dtype=tl.float32)
    Y = tl.tensor(Y, dtype=tl.float32)

    # Get dimensions
    I, J, W = A.shape

    # Initialize variables
    x_n = tl.zeros((W, 1), dtype=tl.float32)
    d_n = tl.zeros((W, 1), dtype=tl.float32)
    p_n = tl.zeros((W, 1), dtype=tl.float32)
    delta_n = delta_0

    # Preprocess A: A = P x1 A (since P is (N x I), we use mode-0 multiplication)
    A = mode_dot(A, P, mode=0)  # A now has shape (N, J, W)

    # Unfold A along mode-2 (mode-3 in MATLAB notation)
    A_unfold = tl.unfold(A, mode=2)  # Shape: (W, N * J)

    # Compute A_T_A and A_Y
    A_T_A = A_unfold @ A_unfold.T  # Shape: (W, W)

    # Reshape Y to match the dimensions
    Y_vector = Y.reshape(-1, 1)  # Shape: (N * J, 1)

    # Compute A_Y
    A_Y = A_unfold @ Y_vector  # Shape: (W, 1)

    # Start iterations
    for n in range(1, max_iter + 1):
        # Update x_n
        # Solve: (A_T_A + delta_n * I) * x_n = A_Y + delta_n * (d_n - p_n)
        left_matrix = A_T_A + delta_n * np.eye(W)
        right_vector = A_Y + delta_n * (d_n - p_n)
        x_n = np.linalg.solve(left_matrix, right_vector)

        # Compute x_hat
        x_hat = lambd * x_n + (1 - lambd) * d_n

        # Update d_n using iterative shrinkage-thresholding
        threshold = epsilon / delta_n
        temp = x_hat + p_n
        d_n = np.maximum(0, temp - threshold) - np.maximum(0, -temp - threshold)

        # Update p_n
        p_n = p_n + x_hat - d_n

        # Update delta_n
        delta_n = min(delta_n, delta_max)

    return x_hat

In [74]:
# Гиперпараметры для компрессивного измерения
max_iter = 100
epsilon_cs = 1e-5  # Используем другое имя для epsilon, чтобы не путать с предыдущим
lambda_ = 0.1
delta_0 = 0.01
delta_max = 1.0

A_cs = tl.tensor(A_tensor, dtype=tl.float32)
Y_cs = tl.tensor(Y, dtype=tl.float32)
P_cs = tl.tensor(P, dtype=tl.float32)

x_hat = tensor_based_compressive_sensing(
    A_cs, P_cs, Y_cs, max_iter, epsilon_cs, lambda_, delta_0, delta_max
)

# Field restoration 𝑋

Используя полученный вектор весов 𝑥, мы можем восстановить поле 𝑋.

In [ ]:
# Восстанавливаем тензорное поле
# x_hat должен быть размерности (R,)
X_reconstructed = tl.tenalg.mode_dot(A_tensor, x_hat.squeeze(), mode=2)  # Применяем x_hat по третьему измерению

# X_reconstructed будет тензором размерности (I, J)